# Criar CNN - Convolutional Neural Network para identificacão de cones com TensorFlow e Keras 

## Desde a instalacao do Anaconda ao Treino e Predicao da rede ( VGG16 )

### Instalando o Anaconda 5.01
Acessar o site
http://anaconda.com/download

Fazer o Download da versao respectiva ao seu SO e também a versao do Python
#### Windows
Python 3.6
https://repo.continuum.io/archive/Anaconda3-5.0.1-Windows-x86_64.exe
Python 2.7
https://repo.continuum.io/archive/Anaconda2-5.0.1-Windows-x86_64.exe
Para instalar é só dar um duplo clique no instalador e seguir 

#### Linux Ubuntu 16.
Python 3.6
https://repo.continuum.io/archive/Anaconda3-5.0.1-Linux-x86_64.sh
Python 2.7
https://repo.continuum.io/archive/Anaconda2-5.0.1-Linux-x86_64.sh
Abrir um terminal, acessar o local onde o arquivo foi baixado e seguir com o seguinte comando
bash ~/Downloads/Anaconda3-5.0.1-Linux-x86_64.sh (versao 3.6 do Python)

A partir deste momento vou cobrir os passos apenas para o linux como sistema operacional

### Instalando o Cuda e Cudnn com suporte a GPU
Caso queira rodar os treinamentos com recursos de GPU é necessário instalar os pacotes CUDA e Cudnn as versoes destes sao muito importantes para que tudo funcione corretamente como até o momento a ultima versao disponível do tensorflow é a 1.4 vou usar as versões de Cuda e Cudnn que a pagina recomenda como versões suportadas lembrando que a placa de video deve ser compatível com cuda e ter os drivers da nvidia instalados
##### Cuda 8.0 GA2
Para facilitar a instalacao vou usar o wget
abrir um terminal acessar o local onde quer que o arquivo seja baixado
    /home/usuario/Downloads
    mkdir Cuda
    cd Cuda
    wget https://developer.nvidia.com/compute/cuda/8.0/Prod2/local_installers/cuda_8.0.61_375.26_linux-run
assim que o download for concluido dar direito de execucao para o arquivo
    sudo chmod +x nomeArquivoBaixado
    sudo ./nomeArquivoBaixado
mesma situacao para o patch desta versao 
    wget https://developer.nvidia.com/compute/cuda/8.0/Prod2/patches/2/cuda_8.0.61.2_linux-run
assim que o download for concluido dar direito de execucao para o arquivo
    sudo chmod +x nomeArquivoPatchBaixado
    sudo ./nomeArquivoPatchBaixado
depois devemos baixar os arquivos do cuda e extrai-los
    wget https://developer.nvidia.com/compute/machine-learning/cudnn/secure/v6/prod/8.0_20170307/Ubuntu16_04_x64/libcudnn6_6.0.20-1+cuda8.0_amd64-deb
    sudo dpkg -i nomeArquivoBaixado

o proximo passo é a inclusao do caminho destes pacotes no PATH
    sudo nano ~/.bashrc
inserir no final do arquivo as seguintes linhas
    export PATH=/usr/local/cuda-8.0/bin${PATH:+:${PATH}}
    export LD_LIBRARY_PATH=/usr/local/cuda-8.0/lib64\
                            ${LD_LIBRARY_PATH:+:${LD_LIBRARY_PATH}}
                            
   sudo apt-get install libcupti-dev

### Instalando os pacotes necessários
#### Criar um ambiente especifico para este tuto
dentro do terminal 
    conda create -n ufes_lcad python=3.6
    source activate ufes_lcad
instalar os pacotes basicos que iremos trabalhar
    conda install keras
    conda install matplotlib
    conda install numpy
    conda install pandas
    conda install scikit-learn
    pip install --ignore-installed --upgrade https://storage.googleapis.com/tensorflow/linux/gpu/tensorflow_gpu-1.4.0-cp36-cp36m-linux_x86_64.whl

### Iniciando o jupyter notebook
dentro do terminal 
    source activate ufes_lcad
    anaconda-navigator ou somente jupyter
clicar em new -> python 3

### Montando a Rede VGG 16 

In [2]:
# Importando os pacotes 
import tensorflow as tf
import keras
import numpy as np
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.optimizers import SGD
from keras.utils import multi_gpu_model
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [3]:
# Definindo variáveis para ajudar nos parametros de treinamento
width, height, batch = 96, 96, 50
sgd = SGD(lr=0.01, decay=5e-4, momentum=0.9, nesterov=True)

In [4]:
# Define que o metodo de montagem do modelo será sequencial camanda por camanda
modelo = Sequential()
# Construindo a primeira camada
# Definimos que esta camada produzirá 64 filtros de saída, seu kernel é de 3 x 3 o stride é de 1 por 1
# a entrada será de 224 por 224 colorida, portanto 3 canais e a funcão de ativacao desta camada será a ReLu 
modelo.add(Conv2D(filters = 64, kernel_size = (3, 3), padding= 'same', strides=(1,1), input_shape = (width, height, 3), activation = 'relu'))
# seguindo sequencialmente configuramos a segunda camada, que assume como entrada a saída da camada anterior e por isso 
# não é preciso declarar este valor, aqui ainda mantemos 4 filtros de saída, kernel de 3 x 3 e stride é de 1 por 1
modelo.add(Conv2D(filters = 64, kernel_size = (3, 3), padding='same', strides=(1,1), activation = 'relu'))
# seguimos para a primeira camada de max Pooling, está camada tem um pool size kernel de 2 por 2 e um stride também de 2 por 2
modelo.add(MaxPooling2D(pool_size = (2, 2), strides=(2,2)))
# com isso concluímos o primeiro bloco da VGG 16, vou considerar o fim de cada bloco sempre após uma camada de pooling

In [5]:
# No segundo bloco da VGG 16 há um aumento no número de saídas das camadas em 2x os outros parametros sao iguais, este
# aumento ocorre sempre que houver um pooling e tem fator de multiplicacao 2 limitado a 512
modelo.add(Conv2D(filters = 128, kernel_size = (3, 3), padding='same', strides=(1,1), activation = 'relu'))
modelo.add(Conv2D(filters = 128, kernel_size = (3, 3), padding='same', strides=(1,1), activation= 'relu'))
# seguimos para a segunda camada de max Pooling, está camada tem um pool size kernel de 2 por 2 e um stride também de 2 por 2
modelo.add(MaxPooling2D(pool_size = (2, 2), strides=(2,2)))

In [6]:
# No terceiro bloco da VGG 16 há um aumento no número de saídas das camadas em 2x e há o acrescimo de mais uma camada 
# convolucional antes do pooling
modelo.add(Conv2D(filters = 256, kernel_size = (3, 3), padding='same', strides=(1,1), activation= 'relu'))
modelo.add(Conv2D(filters = 256, kernel_size = (3, 3), padding='same', strides=(1,1), activation= 'relu'))
modelo.add(Conv2D(filters = 256, kernel_size = (3, 3), padding='same', strides=(1,1), activation= 'relu'))
# seguimos para a terceira camada de max Pooling, está camada tem um pool size kernel de 2 por 2 e um stride também de 2 por 2
modelo.add(MaxPooling2D(pool_size = (2, 2), strides=(2,2)))

In [7]:
# No quarto bloco da VGG 16 há um aumento no número de saídas das camadas em 2x os outros parametros sao iguais
modelo.add(Conv2D(filters = 512, kernel_size = (3, 3), padding='same', strides=(1,1), activation= 'relu'))
modelo.add(Conv2D(filters = 512, kernel_size = (3, 3), padding='same', strides=(1,1), activation= 'relu'))
modelo.add(Conv2D(filters = 512, kernel_size = (3, 3), padding='same', strides=(1,1), activation= 'relu')) 
# seguimos para a terceira camada de max Pooling, está camada tem um pool size kernel de 2 por 2 e um stride também de 2 por 2
modelo.add(MaxPooling2D(pool_size = (2, 2), strides=(2,2)))

In [8]:
# No quinto bloco da VGG 16 todos parametros sao iguais ao da quarta camada, os valores dos filtros nao sao multiplicados por 2 
# porque já atingimos o teto de 512
modelo.add(Conv2D(filters = 512, kernel_size = (3, 3), padding='same', strides=(1,1), activation= 'relu'))
modelo.add(Conv2D(filters = 512, kernel_size = (3, 3), padding='same', strides=(1,1), activation= 'relu'))
modelo.add(Conv2D(filters = 512, kernel_size = (3, 3), padding='same', strides=(1,1), activation= 'relu')) 
# seguimos para a quarta camada de max Pooling, está camada tem um pool size kernel de 2 por 2 e um stride também de 2 por 2
modelo.add(MaxPooling2D(pool_size = (2, 2), strides=(2,2)))

In [9]:
# neste momento preparamos a saída para o formato que a entrada da primeira camada FullyConnected precisa
modelo.add(Flatten())

In [10]:
# serão 3 camadas FC 
# a primeira tem uma entrada de tamanho de 4096 e funcao de ativacao relu
modelo.add(Dense(units = 4096, activation = 'relu'))
# logo após a primeira camada FC é feito um dropout com taxa de 50%
modelo.add(Dropout(0.5))
# a segunda camada FC tem a mesma configuracao da primeira
modelo.add(Dense(units = 4096, activation = 'relu'))
# logo após a segunda camada FC é feito mais um dropout com taxa de 50%
modelo.add(Dropout(0.5))
# a terceira camada na rede original VGG16 tem entrada de tamanho 1000, pois a mesma classifica 1000 classes
# devido a nossa necessidade de classificar apenas 3 classes o meu numero de entrada para a terceira camada será de 3
# a funcao de ativacao da terceira camada a softmax
modelo.add(Dense(units = 3, activation = 'softmax'))

In [11]:
# antes de compilar o modelo defino que o mesmo irá rodar em multiplas GPUs que no meu caso sao 3
#modelo = multi_gpu_model(modelo, gpus=3)
# Seguindo agora para a compilacao do modelo temos como funcao de otimizacao o SGD e a funcao de perda Crossentropy
modelo.compile(optimizer = 'sgd', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [12]:
# antes de iniciar o treino da rede precisamos definir onde estão as fotos que serao usadas para treino
# e onde estao as fotos que serao usadas para teste
# para aumentar o numero de amostras de fotos utilzamos o recurso do keras que aumenta/diminui a foto, inverte, gira e etc
train_datagen = ImageDataGenerator(rescale = 1./255, shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True)
test_datagen = ImageDataGenerator(rescale = 1./255)

# definimos entao onde esta o diretorio contendo o dataset separado em pastas com a seguinte hierarquia
# dentro da pasta dataset temos mais duas pastas com nome treino e teste
# dentro de cada uma das pastas de treino e teste temos a quantidade de pastas equivalente as classes no meu caso 3
# ou seja tenho 3 pastas dentro da pasta treino e mais 3 pastas dentro da pasta teste
# elas devem ter o mesmo nome e conter a classe que será treinada

# aqui definimos também o formato que será a entrada da primeira camada de rede, 
# o padrao de entrada para a VGG16 224 x 224 x 3 
# o batch padrao da VGG16 é de 256
training_set = train_datagen.flow_from_directory('dataset/training_set', target_size = (width, height), batch_size = batch, class_mode = 'categorical')
test_set = test_datagen.flow_from_directory('dataset/test_set', target_size = (width, height), batch_size = batch, class_mode = 'categorical')

Found 8591 images belonging to 3 classes.
Found 2200 images belonging to 3 classes.


In [32]:
# antes de iniciar o treino podemos exibir como está montada a rede
modelo.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_16 (Conv2D)           (None, 224, 224, 64)      1792      
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 224, 224, 64)      36928     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 112, 112, 64)      0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 112, 112, 128)     73856     
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 112, 112, 128)     147584    
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 56, 56, 128)       0         
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 56, 56, 256)       295168    
__________

In [13]:
# podemos agora iniciar o treinamento da nossa rede
# aqui ainda definimos o numero de passos por treino e validacao e tambem a quantidade de epochs
modelo.fit_generator(training_set, steps_per_epoch = 8591, epochs = 15, validation_data = test_set, validation_steps = 2200,  use_multiprocessing=True)


Epoch 1/15
8591/8591 [==============================] - 1742s 203ms/step - loss: 0.7207 - acc: 0.5983 - val_loss: 0.5243 - val_acc: 0.7432
Epoch 2/15
8591/8591 [==============================] - 1739s 202ms/step - loss: 0.4056 - acc: 0.8233 - val_loss: 0.2860 - val_acc: 0.8845
Epoch 3/15
8591/8591 [==============================] - 1739s 202ms/step - loss: 0.1495 - acc: 0.9432 - val_loss: 0.3093 - val_acc: 0.9159
Epoch 4/15
8591/8591 [==============================] - 1739s 202ms/step - loss: 0.0463 - acc: 0.9838 - val_loss: 0.2839 - val_acc: 0.9268
Epoch 5/15
8591/8591 [==============================] - 1739s 202ms/step - loss: 0.0204 - acc: 0.9931 - val_loss: 0.3837 - val_acc: 0.9305
Epoch 6/15
8591/8591 [==============================] - 1739s 202ms/step - loss: 0.0161 - acc: 0.9946 - val_loss: 0.3595 - val_acc: 0.9291
Epoch 7/15
8591/8591 [==============================] - 1739s 202ms/step - loss: 0.0070 - acc: 0.9977 - val_loss: 0.3848 - val_acc: 0.9382
Epoch 8/15
8591/8591 [=====

In [15]:
from keras.preprocessing import image
test_image = image.load_img('dataset/single_prediction/cat_or_dog_2.jpg', target_size = (96, 96))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = modelo.predict(test_image)
print(result)
training_set.class_indices

[[ 0.  1.  0.]]


{'cats': 0, 'cone': 1, 'dogs': 2}

In [17]:
## Save model pesos
modelo.save('VGG16.h5')
print("Model saved")    
## Save model estrutura
modelo_json = modelo.to_json()
with open("VGG16Estrutura.json", "w") as json_file:
    json_file.write(modelo_json)
    

Model saved
